In [2]:
%%shell
pip install cython
pip list | grep -P 'Cython'

Cython                           0.29.36


In [30]:
import numpy as np

In [3]:
%load_ext Cython

In [4]:
%%timeit
x = 1
x += 1

43.3 ns ± 1.16 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


$$
f_i = f_{i-1} + f_{i-2}
$$

In [6]:
def fib(n: int) -> int:
    a, b = 1, 1
    for i in range(2, n + 1):
        a, b = b, a + b
    return b

In [13]:
%%timeit
fib(100)

8.34 µs ± 2.87 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [14]:
%%cython
def fib(n: int) -> int:
    a, b = 1, 1
    for i in range(2, n + 1):
        a, b = b, a + b
    return b

In [15]:
fib

<function _cython_magic_21c9094ba3555ad62d6d64430c2f878e.fib>

In [17]:
%%timeit
fib(100)

4.12 µs ± 1.32 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [27]:
%%cython
cpdef long int fib(long int n):
    cdef long int a = 1, b = 1, i
    for i in range(2, n + 1):
        a, b = b, a + b
    return b

In [29]:
%%timeit
fib(100)

156 ns ± 25.4 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [32]:
y1 = np.random.randint(0, 2, size=(1000,))
y2 = np.random.randint(0, 2, size=(1000,))

In [33]:
from sklearn.metrics import confusion_matrix

In [34]:
cm = confusion_matrix(y1, y2)

In [35]:
y1[:10]

array([1, 1, 0, 0, 1, 0, 1, 1, 0, 0])

In [36]:
y2[:10]

array([0, 1, 1, 0, 1, 0, 1, 1, 1, 1])

In [37]:
cm

array([[240, 248],
       [260, 252]])

In [38]:
confusion_matrix?

In [41]:
import numpy as np

def confusion_matrix_cython(
    y1, y2
    ):
    n_classes = np.unique(y1).size
    cm = np.zeros((n_classes, n_classes))
    for i in range(y1.size):
        cm[y1[i], y2[i]] += 1
    return cm

In [42]:
confusion_matrix_cython(y1, y2)

array([[240., 248.],
       [260., 252.]])

In [43]:
%%timeit
confusion_matrix(y1, y2)

888 µs ± 217 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [44]:
%%timeit
confusion_matrix_cython(y1, y2)

684 µs ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [57]:
%%cython
import numpy as np
cimport numpy as np
cimport cython

np.import_array()

DTYPE = np.int
ctypedef np.int_t DTYPE_t

cpdef np.ndarray[DTYPE_t, ndim=2] confusion_matrix_cython(
    np.ndarray[DTYPE_t, ndim=1] y1,
    np.ndarray[DTYPE_t, ndim=1] y2
    ):
    cdef int n_classes = np.unique(y1).size, i
    cdef np.ndarray[DTYPE_t, ndim=2] cm = np.zeros((n_classes, n_classes), dtype=DTYPE)
    for i in range(y1.size):
        cm[y1[i], y2[i]] += 1
    return cm

In [58]:
y2.dtype

dtype('int64')

In [59]:
confusion_matrix_cython(y1, y2)

array([[240, 248],
       [260, 252]])

In [60]:
%%timeit
confusion_matrix_cython(y1, y2)

22.6 µs ± 2.71 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [91]:
from numba import njit, int64

In [95]:
@njit(int64[:, :](int64[:], int64[:]))
def confusion_matrix_numba(
    y1: np.ndarray, y2: np.ndarray
    ) -> np.ndarray:
    n_classes = np.unique(y1).size
    cm = np.zeros((n_classes, n_classes), dtype=np.int64)
    for i in range(y1.size):
        cm[y1[i], y2[i]] += 1
    return cm

In [96]:
confusion_matrix_numba(y1, y2)

array([[240, 248],
       [260, 252]])

In [98]:
%%timeit
confusion_matrix_numba(y1, y2)

21.5 µs ± 4.6 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [66]:
%%writefile main.py
def fib(n: int) -> int:
    a, b = 1, 1
    for i in range(2, n + 1):
        a, b = b, a + b
    return b

Overwriting main.py


In [68]:
from main import fib

In [69]:
fib(100)

573147844013817084101

In [85]:
%%writefile main2.pyx
# cython: language_level=3
cpdef long long int fib(long long int n):
    cdef long long int a = 1, b = 1, i
    for i in range(2, n + 1):
        a, b = b, a + b
    return b


Overwriting main2.pyx


In [86]:
%%shell
cython main2.pyx
ls

build				       main.cpython-310-x86_64-linux-gnu.so
main2.c				       main.py
main2.cpython-310-x86_64-linux-gnu.so  main.pyx
main2.pyx			       __pycache__
main.c				       sample_data


In [87]:
%%shell
cythonize -i main2.pyx
ls

build				       main.cpython-310-x86_64-linux-gnu.so
main2.c				       main.py
main2.cpython-310-x86_64-linux-gnu.so  main.pyx
main2.pyx			       __pycache__
main.c				       sample_data


In [88]:
from main2 import fib

In [89]:
fib(100)

573147844013817084101

In [90]:
888 / 22.6

39.292035398230084